# SVM Ranking Demo
* This demo is a simulation. Imagine a list of N items [1, 2, 3, ... , N] which we know the ground truth ranking for.
* Each item is simply ranked by its index or number.
* Now lets randomly show an SVC a subset of all possible comparisons and let the SVC guess the rankings.
* How good is the generated ranking?

# Generate Data 

In [119]:
from random import random
import numpy as np

In [120]:
N   = 100          # Item count
PR  = 10           # Comparisons per item
NOS = 0.05         # Noise ratio
CPP = 3            # Comparison per pair
TOT = N * PR * CPP # Total # of pairs

In [121]:
# Generate all possible unique pairs and then subsample

# Generate all n * ( n - 1 ) / 2
poss_pairs = [(x, y) for x in range(N) for \
                         y in range(N) if x < y]

# Sample N * PR (items * pairs_per_item) from the total
pairs = rand.sample(poss_pairs, N * PR) * CPP
pairs = np.array(pairs)

pairs[:5]

array([[37, 68],
       [39, 52],
       [30, 33],
       [56, 98],
       [20, 42]])

# Convert to One-hot

In [122]:
# One-hot encoding

# Function to one-hot encode pairs:
# (0, 3) => [1, 0, 0 ,0] - [0, 0, 0, 1] = [1, 0, 0, -1]
idy = np.identity(N)
vectorise = lambda x:idy[x[0]] - idy[x[1]]

# Encode
X = np.array(list(map(vectorise, pairs)))

X[:5]

array([[ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0., -1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        -1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0

# Add Noise

In [123]:
# Add noise
y = [1 if (random() > NOS) else -1 for i in range(TOT)]
y = np.array(y).reshape(TOT, 1)

y[:10]

array([[ 1],
       [ 1],
       [ 1],
       [ 1],
       [ 1],
       [ 1],
       [ 1],
       [ 1],
       [ 1],
       [-1]])

# Balance Class Labels

In [124]:
# Balance class labels by randomly flipping comparison direction
flip = [1 if (random() > 0.5) else -1 for i in range(TOT)]
flip = np.array(flip).reshape(TOT, 1)

# Flip the pairs
X = np.multiply(flip, X)
y = np.multiply(flip, y)

X[:5]

array([[-0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0.,
        -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0.,
        -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -1., -0.,
        -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0.,
        -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0.,
        -0., -0., -0.,  1., -0., -0., -0., -0., -0., -0., -0., -0., -0.,
        -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0.,
        -0., -0., -0., -0., -0., -0., -0., -0., -0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        -1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0

# Rank using SVC

In [125]:
from sklearn.svm import LinearSVC

In [126]:
def SVC_rank(X, Y):
    cls = LinearSVC(C=0.1)
    model = cls.fit(X, Y)
    
    res = model.coef_
    ranks = np.flip(np.argsort(res), axis=1)
    
    return ranks

In [127]:
SVC_rank(X, y)

/Users/nhalabi/anaconda/lib/python3.6/site-packages/sklearn/utils/validation.py:526: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


array([[ 1,  0,  4,  7,  2, 12,  6,  3, 20, 24,  9, 14, 11,  8,  5, 13, 10,
        17, 28, 21, 25, 15, 26, 31, 16, 23, 19, 30, 18, 27, 29, 34, 36, 47,
        32, 37, 22, 33, 42, 39, 51, 48, 50, 38, 43, 35, 41, 46, 49, 45, 63,
        44, 54, 52, 40, 64, 60, 55, 53, 59, 61, 56, 57, 65, 58, 73, 66, 74,
        67, 76, 71, 69, 70, 62, 72, 75, 79, 89, 77, 78, 85, 92, 88, 83, 86,
        84, 80, 81, 68, 87, 93, 91, 90, 94, 82, 98, 96, 95, 99, 97]])

In [128]:
from scipy.stats import spearmanr
spearmanr(SVC_rank(X, y)[0], range(N))

/Users/nhalabi/anaconda/lib/python3.6/site-packages/sklearn/utils/validation.py:526: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


SpearmanrResult(correlation=0.97671767176717661, pvalue=2.5166475834094379e-67)

# Motivation
* The applications (Recommender systems and IR).
* System comparison (Can we do statistical significance?).
* Simulation can be improved.
* The above can be improved both in ranking quality and time complexity.

# Demotivation
* Does not scale well.